In [1]:
from validate_coarse_to_fine import * 
from dataloader.BSD500 import BSD500
import torch
from R_network_mm import RNet
import json
from matplotlib import pyplot as plt
from torchmetrics.functional import peak_signal_noise_ratio as psnr
from torchmetrics.functional import structural_similarity_index_measure as ssim

In [2]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

def compute_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = compare_psnr(Iclean[0,:,:,:], Img[0,:,:,:], data_range=data_range)
    return PSNR
    

In [3]:
test_dataset = BSD500('data/test.h5')

In [4]:
sigma = 25.0  

val_data = pd.read_csv(f"{'denoise_val_results'}/validation_scores_{'sigma_25_mm_valid'}.csv").reset_index(drop=True)
p1 = val_data.loc[val_data["psnr"].idxmax()]["p1"]

exp = "/home/pourya/mm_final/exps/sigma25/64_2_7_1e-3_majorized"
path_ckp = exp + "/checkpoints/checkpoint_best_epoch.pth"
path_config = exp + "/config.json"
device = 'cuda:2'
config = json.load(open(path_config))
ckp = torch.load(path_ckp, map_location={'cuda:0':device,'cuda:1':device,'cuda:2':device,'cuda:3':device})

model = RNet(config['model_params'])
model.to(device)
model.load_state_dict(ckp['state_dict'])
model.eval()

model.W2.W1.weight.data = model.sumtoone(model.W2.W1.weight)
for i, _ in enumerate(model.W2.W1s): 
    model.W2.W1s[i].weight.data = model.sumtoone(model.W2.W1s[i].weight)


model.W1.W1.weight.data = model.zeromean(model.W1.W1.weight)
for i, _ in enumerate(model.W1.W1s): 
    model.W1.W1s[i].weight.data = model.zeromean(model.W1.W1s[i].weight)


model.eigenimage = model.eigenimage.to(device)
print(model.lmbda, p1)
lip, model.eigenimage = model.W1.cal_lip(model.eigenimage, 1000)
model.alpha = 1 / lip
model.lmbda = torch.nn.Parameter(torch.ones(1, 1).to(device) * p1)


Parameter containing:
tensor([[0.0011]], device='cuda:2', requires_grad=True) 0.0011094470382648


In [5]:
def mmr_denoise(y):
    s = 1e-3
    n = 5
    e = 1e-5
    q = (e/s) ** (1/n)
    tols = [s * q**(i) for i in range(n+1)]
    n_out = 10
    for _ in range(n_out-(n+1)):
        tols.append(e)
    with torch.no_grad(): 
        n_in = 100  
        c_k = model.prox_denoise_no_mask(y, y, n_in, tols[0], check_tol=True)
        for it in range(n_out - 1):   
            model.cal_mask(c_k)
            c_k = model.prox_denoise_with_mask(y, c_k, n_in, tols[it+1], check_tol=True)

    return c_k

In [6]:
i = 0
psnrs_all = list()

with torch.no_grad():
    for runs in range(5):
        psnrs = list()
        for img in test_dataset:
            i = i + 1
            if True:
                gt = img.to(device)[None, :, :, :]
                noisy_image = (img.to(device) + (25/255.0)*torch.randn(img.shape, device=device))[None, :, :, :]
                denoised =mmr_denoise(noisy_image)
                psnr = np.round(compute_PSNR(denoised, gt, 1), 2)
                psnrs.append(psnr)
                print(i, psnr)
                
        psnrs_all.append(psnrs)

1 29.65
2 28.18
3 30.24
4 29.46
5 29.46
6 26.24
7 28.3
8 28.2
9 29.19
10 28.6
11 27.71
12 25.85
13 29.2
14 26.36
15 27.4
16 30.29
17 27.98
18 28.77
19 31.0
20 28.95
21 32.54
22 26.71
23 26.43
24 24.12
25 27.52
26 28.33
27 27.24
28 32.45
29 25.67
30 29.73
31 27.36
32 31.59
33 29.42
34 23.67
35 28.16
36 27.88
37 28.59
38 27.27
39 32.79
40 25.9
41 30.43
42 31.91
43 30.48
44 29.78
45 26.61
46 29.42
47 28.66
48 26.56
49 25.61
50 37.54
51 27.18
52 33.13
53 32.59
54 29.79
55 28.62
56 26.96
57 25.74
58 27.01
59 24.32
60 26.36
61 27.97
62 27.63
63 30.4
64 24.53
65 28.38
66 32.07
67 32.36
68 29.13
69 29.66
70 28.16
71 30.27
72 29.42
73 29.47
74 26.25
75 28.25
76 28.26
77 29.16
78 28.58
79 27.7
80 25.87
81 29.2
82 26.36
83 27.42
84 30.27
85 27.97
86 28.75
87 31.03
88 28.96
89 32.56
90 26.69
91 26.41
92 24.12
93 27.49
94 28.37
95 27.22
96 32.54
97 25.68
98 29.75
99 27.37
100 31.64
101 29.41
102 23.69
103 28.19
104 27.89
105 28.6
106 27.24
107 32.73
108 25.88
109 30.42
110 31.93
111 30.44
112 29.77

In [8]:
psnr_mat = np.zeros((5, 68))

for i in range(5):
    psnr_mat[i, :] = np.array(psnrs_all[i])

std_vec = np.std(psnr_mat, axis=0)
avg_vec = np.mean(psnr_mat, axis=0)

print('std mean:', np.mean(std_vec))
print('mean mean:', np.mean(avg_vec))

std mean: 0.028233295005375622
mean mean: 28.606529411764704
